<a href="https://colab.research.google.com/github/SohanRaidev/FreshCheckAI/blob/main/freshness_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow keras opencv-python scikit-learn


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!unzip '/content/drive/MyDrive/freshness detection/archive.zip' -d '/content/dataset'


Streaming output truncated to the last 5000 lines.
  inflating: /content/dataset/dataset/Train/rottencapsicum/WhatsApp Image 2020-11-07 at 11.43.17 PM.jpg_0_5372.jpg  
  inflating: /content/dataset/dataset/Train/rottencapsicum/WhatsApp Image 2020-11-07 at 11.43.17 PM.jpg_0_54.jpg  
  inflating: /content/dataset/dataset/Train/rottencapsicum/WhatsApp Image 2020-11-07 at 11.43.17 PM.jpg_0_6186.jpg  
  inflating: /content/dataset/dataset/Train/rottencapsicum/WhatsApp Image 2020-11-07 at 11.43.17 PM.jpg_0_6285.jpg  
  inflating: /content/dataset/dataset/Train/rottencapsicum/WhatsApp Image 2020-11-07 at 11.43.17 PM.jpg_0_6551.jpg  
  inflating: /content/dataset/dataset/Train/rottencapsicum/WhatsApp Image 2020-11-07 at 11.43.17 PM.jpg_0_6565.jpg  
  inflating: /content/dataset/dataset/Train/rottencapsicum/WhatsApp Image 2020-11-07 at 11.43.17 PM.jpg_0_6985.jpg  
  inflating: /content/dataset/dataset/Train/rottencapsicum/WhatsApp Image 2020-11-07 at 11.43.17 PM.jpg_0_7173.jpg  
  inflating: /c

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator


data_dir = '/content/dataset/dataset'

# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

# Training data
train_generator = train_datagen.flow_from_directory(
    data_dir + '/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training')

# Validation data
validation_generator = train_datagen.flow_from_directory(
    data_dir + '/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation')


Found 18901 images belonging to 18 classes.
Found 4718 images belonging to 18 classes.


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
from tensorflow.keras import backend as K


K.clear_session()


input_shape = (224, 224, 3)
num_classes = 18


inputs = layers.Input(shape=input_shape)


base_model = VGG16(weights='imagenet', include_top=False, input_tensor=inputs)


base_model.trainable = False


x = base_model.output
x = layers.Flatten()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)

model = models.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │       6,422,784 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 21,142,098 (80.65 MB)

 Trainable params: 6,427,410 (24.52 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint('/content/drive/MyDrive/freshness-detection/model/saved_modelpro.keras', # Changed .h5 to .keras
                             monitor='val_accuracy',
                             save_best_only=True)

history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=20,
    callbacks=[early_stopping, checkpoint]
)

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


591/591 ━━━━━━━━━━━━━━━━━━━━ 417s 671ms/step - accuracy: 0.3511 - loss: 2.1906 - val_accuracy: 0.6195 - val_loss: 1.0997
Epoch 2/20
591/591 ━━━━━━━━━━━━━━━━━━━━ 365s 612ms/step - accuracy: 0.5807 - loss: 1.1408 - val_accuracy: 0.6482 - val_loss: 1.0092
Epoch 3/20
591/591 ━━━━━━━━━━━━━━━━━━━━ 362s 608ms/step - accuracy: 0.6178 - loss: 1.0185 - val_accuracy: 0.6477 - val_loss: 0.8585
Epoch 4/20
591/591 ━━━━━━━━━━━━━━━━━━━━ 360s 605ms/step - accuracy: 0.6382 - loss: 0.9590 - val_accuracy: 0.6602 - val_loss: 0.9731
Epoch 5/20
549/591 ━━━━━━━━━━━━━━━━━━━━ 20s 488ms/step - accuracy: 0.6602 - loss: 0.8866

In [ ]:
# Evaluate model performance on validation data
val_loss, val_acc = model.evaluate(validation_generator)
print(f"Validation Accuracy: {val_acc}")


69/69 ━━━━━━━━━━━━━━━━━━━━ 35s 500ms/step - accuracy: 0.9866 - loss: 0.0376
Validation Accuracy: 0.9871441721916199


In [ ]:
# Save the trained model to Google Drive
model.save('/content/drive/My Drive/freshnessrecognitionpro.keras')


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
from PIL import UnidentifiedImageError


model = load_model('/content/drive/MyDrive/freshnessrecognition.keras')


def preprocess_image(img_path):
    try:
        img = image.load_img(img_path, target_size=(224, 224))
    except UnidentifiedImageError:
        print(f"Error: Unable to load image at {img_path}")
        return None
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array / 255.0


def predict_freshness(img_path, class_labels):
    img = preprocess_image(img_path)
    if img is None:
        return
    prediction = model.predict(img)
    class_index = np.argmax(prediction)
    predicted_class = class_labels[class_index]
    print(f"Predicted Class: {predicted_class}")

class_labels = {
    0: 'freshapples',
    1: 'freshbanana',
    2: 'freshorange',
    3: 'rottenapple',
    4: 'rottenbanana',
    5: 'rottenorange'
}

# Test with a new image (replace with the actual path to your test image)
img_path = '/content/dataset/dataset/dataset/test/freshapples/frame_1010.jpg'  # Adjust path as needed
predict_freshness(img_path, class_labels)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Predicted Class: freshapples


In [ ]:
#Camera Capture Method(capture image)

from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
import cv2
import PIL
from PIL import Image
import io

# JavaScript code to capture image from the webcam
def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
        async function takePhoto(quality) {
            const div = document.createElement('div');
            const capture = document.createElement('button');
            capture.textContent = 'Capture';
            div.appendChild(capture);

            const video = document.createElement('video');
            video.style.display = 'block';
            const stream = await navigator.mediaDevices.getUserMedia({video: true});
            document.body.appendChild(div);
            div.appendChild(video);
            video.srcObject = stream;
            await video.play();

            google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

            await new Promise((resolve) => capture.onclick = resolve);

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getTracks().forEach(track => track.stop());
            div.remove();
            return canvas.toDataURL('image/jpeg', quality);
        }
        ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    return filename

# Capture the image
image_file = take_photo()
print(f"Image saved as {image_file}")


<IPython.core.display.Javascript object>

Image saved as photo.jpg


In [ ]:
#Camera Capture Method(run from captured image.)
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))  # Resize to 224x224
    img_array = image.img_to_array(img)  # Convert the image to an array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array / 255.0  # Normalize the image

def predict_freshness(img_path, class_labels):
    img = preprocess_image(img_path)
    prediction = model.predict(img)
    class_index = np.argmax(prediction)
    predicted_class = class_labels[class_index]
    print(f"Predicted Class: {predicted_class}")


class_labels = {
    0: 'freshapples',
    1: 'freshbanana',
    2: 'freshorange',
    3: 'rottenapple',
    4: 'rottenbanana',
    5: 'rottenorange'
}

# Run the prediction on the captured image
predict_freshness('photo.jpg', class_labels)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 572ms/step
Predicted Class: rottenbanana


In [ ]:
#CODE TO MAKE DATASET BY CAPTURING FRAME BY FRAME SHOT FROM A VIDEO.

import cv2
import os

# Function to extract frames from video and save them as images
def extract_frames(video_path, output_folder, frame_rate=30):
    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Capture the video
    cap = cv2.VideoCapture(video_path)

    # Get the total number of frames in the video
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Frame extraction loop
    frame_number = 0
    while True:
        ret, frame = cap.read()  # Read frame-by-frame
        if not ret:
            break

        # Save every nth frame (e.g., every 30th frame)
        if frame_number % frame_rate == 0:
            frame_filename = os.path.join(output_folder, f'frame_{frame_number}.jpg')
            cv2.imwrite(frame_filename, frame)
            print(f"Saved {frame_filename}")

        frame_number += 1


    cap.release()
    print(f"Extracted {frame_number} frames from the video.")

# Example usage:
video_path = '/content/drive/MyDrive/apple2.mp4'
output_folder = '/content/dataset/dataset/dataset/test/freshapples'  # Folder where you want to save the frames
extract_frames(video_path, output_folder, frame_rate=10)  # Adjust frame_rate to extract every 30th frame


Saved /content/dataset/dataset/dataset/test/freshapples/frame_0.jpg
Saved /content/dataset/dataset/dataset/test/freshapples/frame_10.jpg
Saved /content/dataset/dataset/dataset/test/freshapples/frame_20.jpg
Saved /content/dataset/dataset/dataset/test/freshapples/frame_30.jpg
Saved /content/dataset/dataset/dataset/test/freshapples/frame_40.jpg
Saved /content/dataset/dataset/dataset/test/freshapples/frame_50.jpg
Saved /content/dataset/dataset/dataset/test/freshapples/frame_60.jpg
Saved /content/dataset/dataset/dataset/test/freshapples/frame_70.jpg
Saved /content/dataset/dataset/dataset/test/freshapples/frame_80.jpg
Saved /content/dataset/dataset/dataset/test/freshapples/frame_90.jpg
Saved /content/dataset/dataset/dataset/test/freshapples/frame_100.jpg
Saved /content/dataset/dataset/dataset/test/freshapples/frame_110.jpg
Saved /content/dataset/dataset/dataset/test/freshapples/frame_120.jpg
Saved /content/dataset/dataset/dataset/test/freshapples/frame_130.jpg
Saved /content/dataset/dataset/